In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
import time
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e6/sample_submission.csv
/kaggle/input/playground-series-s5e6/train.csv
/kaggle/input/playground-series-s5e6/test.csv


In [2]:
#train the data
train_df = pd.read_csv("/kaggle/input/playground-series-s5e6/train.csv")
test_df = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')
sample_submission_csv = pd.read_csv('/kaggle/input/playground-series-s5e6/sample_submission.csv')

In [3]:
print(train_df.shape)
print(test_df.shape)
train_df.head()

(750000, 10)
(250000, 9)


,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,0,37,70,36,Clayey,Sugarcane,36,4,5,28-28
1,1,27,69,65,Sandy,Millets,30,6,18,28-28
2,2,29,63,32,Sandy,Millets,24,12,16,17-17-17
3,3,35,62,54,Sandy,Barley,39,12,4,10-26-26
4,4,35,58,43,Red,Paddy,37,2,16,DAP


In [4]:
x = train_df.drop(columns=['Fertilizer Name', 'id'])
y = train_df['Fertilizer Name']
x_test = test_df.drop(columns=['id'])

In [5]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [6]:
le_soil = LabelEncoder()
x['Soil Type'] = le_soil.fit_transform(x['Soil Type']).astype(int)
x_test['Soil Type'] = le_soil.transform(x_test['Soil Type']).astype(int)
le_crop = LabelEncoder()
x['Crop Type'] = le_crop.fit_transform(x['Crop Type']).astype(int)
x_test['Crop Type'] = le_crop.transform(x_test['Crop Type']).astype(int)

In [7]:
model = XGBClassifier(random_state=42,
                      n_estimators=1000,
                      learning_rate=0.03,
                      max_depth=7,
                      subsample=0.8,
                      colsample_bytree=0.3,
                      reg_alpha=0.5,
                      reg_lambda=1.0,
                      objective='multi:softprob',
                      use_label_encoder=False,
                      num_class = len(np.unique(y)),
                      max_bin = 128,
                      colsample_bylevel=1,
                      colsample_bynode=1,
                      tree_method = 'hist',
                      device = 'cuda',
                      enable_categorical = True,
                      early_stopping_rounds = 50,
                    eval_metric='mlogloss')

In [8]:
model.fit(x, y_encoded,eval_set = [(x,y_encoded)],verbose=10)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [14:04:43] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)


[0]	validation_0-mlogloss:1.94567
[10]	validation_0-mlogloss:1.94347
[20]	validation_0-mlogloss:1.94158
[30]	validation_0-mlogloss:1.93997
[40]	validation_0-mlogloss:1.93860
[50]	validation_0-mlogloss:1.93731
[60]	validation_0-mlogloss:1.93619
[70]	validation_0-mlogloss:1.93508
[80]	validation_0-mlogloss:1.93409
[90]	validation_0-mlogloss:1.93321
[100]	validation_0-mlogloss:1.93236
[110]	validation_0-mlogloss:1.93153
[120]	validation_0-mlogloss:1.93069
[130]	validation_0-mlogloss:1.92991
[140]	validation_0-mlogloss:1.92918
[150]	validation_0-mlogloss:1.92851
[160]	validation_0-mlogloss:1.92785
[170]	validation_0-mlogloss:1.92718
[180]	validation_0-mlogloss:1.92654
[190]	validation_0-mlogloss:1.92587
[200]	validation_0-mlogloss:1.92531
[210]	validation_0-mlogloss:1.92472
[220]	validation_0-mlogloss:1.92415
[230]	validation_0-mlogloss:1.92363
[240]	validation_0-mlogloss:1.92311
[250]	validation_0-mlogloss:1.92258
[260]	validation_0-mlogloss:1.92211
[270]	validation_0-mlogloss:1.92161
[28

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.3,
              device='cuda', early_stopping_rounds=50, enable_categorical=True,
              eval_metric='mlogloss', feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.03, max_bin=128,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None, num_class=7,
              num_parallel_tree=None, ...)

In [9]:
start_time = time.time()
training_time = time.time() - start_time

In [10]:
def map3(y_true, y_pred_probs):
    y_true = [[x] for x in y_true]
    y_pred_probs = np.argsort(y_pred_probs, axis=1)[:, -3:][:, ::-1].tolist()
    
    def ap3(y_true, y_pred_probs):
        y_pred_probs = y_pred_probs[:3]

        score = 0.0
        num_hits = 0.0

        for i,p in enumerate(y_pred_probs):
            if p in y_true and p not in y_pred_probs[:i]:
                num_hits += 1.0
                score += num_hits / (i+1.0)

        if not y_true:
            return 0.0

        return score
    
    return np.mean([ap3(a,p) for a,p in zip(y_true, y_pred_probs)])

In [11]:
sample_size = min(5000,len(x))
sample_idx = np.random.choice(len(x), sample_size,replace=False)
sample_probs = model.predict_proba(x.iloc[sample_idx])
sample_map3 = map3(y_encoded[sample_idx], sample_probs)

prediction_start = time.time()
test_prob = model.predict_proba(x_test)


In [12]:
top3 = np.argsort(test_prob, axis=1)[:,-3:][:,::-1]
top3_fertilizers = []
for row in top3:
    fertilizer_names = le.inverse_transform(row)
    top3_fertilizers.append(' '.join(fertilizer_names))

In [13]:
prediction_time = time.time() - prediction_start

In [14]:
submission = pd.DataFrame({
    'id': test_df['id'],
    'Fertilizer Name': top3_fertilizers
})
submission.to_csv('submission.csv', index=False)